In [15]:
import pandas as pd
import requests
import time
import os
from dotenv import load_dotenv

# 📂 .env 파일에서 API 키 읽어오기
load_dotenv()
REST_API_KEY = os.getenv('KAKAO_API_KEY')

# ✅ 광진구 어린이 보호구역 파일 경로
child_zone_path = '../../../data/gwangjin_child_zone_cp949.csv'
child_df = pd.read_csv(child_zone_path, encoding='cp949')

# 📍 결과 저장할 리스트
latitudes = []
longitudes = []

# 📍 카카오 주소→좌표 변환 함수
def get_lat_lon(address, api_key):
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        result = response.json()["documents"]
        if result:
            return float(result[0]["y"]), float(result[0]["x"])  # 위도, 경도
        else:
            return None, None
    else:
        return None, None

# 📍 어린이 보호구역 주소 하나씩 위경도 변환
for address in child_df['도로명주소']:  # 도로명주소 컬럼명 맞춰야 함
    lat, lon = get_lat_lon(address, REST_API_KEY)
    latitudes.append(lat)
    longitudes.append(lon)
    time.sleep(0.1)  # 요청 속도 너무 빠르면 막힐 수 있어서 살짝 쉬기

# 📍 결과 추가
child_df['위도'] = latitudes
child_df['경도'] = longitudes

# 📂 위경도 추가된 파일로 저장 (CP949로 다시 저장)
child_df.to_csv('../../../data/gwangjin_child_zone_with_coords.csv', index=False, encoding='cp949')

print("✅ 변환 완료! 'gwangjin_child_zone_with_coords.csv' 저장 완료")


ModuleNotFoundError: No module named 'dotenv'

In [11]:
import pandas as pd

# 📂 UTF-8로 읽기
child_df = pd.read_csv('../../../data/gwangjin_child_zone.csv', encoding='utf-8')

# (여기서 필요한 작업을 한다면 추가)

# 📂 CP949로 저장하기
child_df.to_csv('../../../data/gwangjin_child_zone_cp949.csv', index=False, encoding='cp949')

print("✅ 변환 완료: 'gwangjin_child_zone_cp949.csv' (cp949 인코딩)")


✅ 변환 완료: 'gwangjin_child_zone_cp949.csv' (cp949 인코딩)
